# General

* This notebook contains the implementation of:
    * A ```GreedyBlockAllocator``` that implements a collective-preferring greedy allocation heuristic to pack partitioned graphs into a (sub-set of a) RAMP topology
    * A ```Partition``` class that will create a partitioned version of a DAG based on the original file provided by pipedream
    
* The notebook first introduces the ```GreedyBlockAllocator``` class with pseudo-code, then the ```Partition``` class with pseudo-code, then some real code showing how they are used together (bottom of the notebook)

# GreedyBlockAllocator: Job allocation in RAMP

## Given:

* An un-partitioned and non-mirrored job-graph,
```python
job_graph = nx.DiGraph()
```
where
```python
job_graph.nodes[node] = {'activation':int/float, 'parameters':int/float}
```

* A list containing the IDs of the nodes that are going to be split (maintaining the pipedream ID convention)
```python
mp_split_ids = ['1','2','5',...]
```

* A list containing how many times each op will be split, with indices corresponding to those from ```mp_split_ids``` above
```python
mp_splits = [2,2,4,...]
```
For example this means that node '1' will be split 2 times, node '2' 2 times, node '5' 4 times and so on.

* A RAMP topology where each node is identified by its unique ```(communication-group, rack, server)``` 3-tuple identifier and records it's available resourcs and which operations are on it, as well as a 3-tuple indicating the 'shape' of this RAMP topology w.r.t. the dimensions of RAMP
```python
ramp_topology = {(1,1,1):{'activation':X,'ops':['1a','4b']},...}
ramp_shape = (2,4,2)
```

## Do an allocation:

* Get some allocation 'preamble' (information related to the job to be allocated that's useful for the allocation heuristic)
```python
sequence, splits, op_server_info, parents, children = GreedyBlockAllocator.get_allocation_preamble(job_graph,mp_split_ids,mp_splits)
```
where
    * ```sequence``` is an ordered (using topological sort) list specifying the order that ops can be greedily placed (i.e. start with ops that have no children, then do their children and so on)
    * ```splits``` is an ordered list specifying the number of splits per op, with indices corresponding to ```sequence```
    * ```op_server_info``` is a dict maintaining information about which servers are being used for which ops (i.e. ```{'1':[(1,1,1),(1,1,2)],...}``` specifies that op '1' is split across the servers (1,1,1) and (1,1,2)). This used to check where an ops parent(s) are to see if it can be placed across the same set of servers.
    * ```parents``` is a dict relating parent ops to their children (i.e. ```{'1':['2','5'],...}``` indicates that ops 2 and 5 are both children of op 1).
    * ```children``` is a dict relating child ops to their parents (i.e. ```{'2':['1'],...}``` indicates that op 1 is a parent of op 2).

* Specify the 3-D (w.r.t. RAMP dimensions) shape of the 'meta-block'. The 'meta-block' is the set of servers that will be reserved for a job, where following this reservation the packing heuristic will try to place all ops within this 'meta-block'
```python
meta_block_shape = (2,4,2)
```
Note that none of the dimensions of the meta-block can be larger than the dimensions of the RAMP topology in use (otherwise it would be asking for resources that do not exist within that RAMP topology)
    * NOTE: this shape is what should be output by an agent

* Find a specific set of servers in the RAMP topology that can be reserved for the meta-block. Servers can only be reserved if there are no ops currently using them (i.e. different jobs cannot share servers).
```python
meta_block_info = GreedyBlockAllocator.find_meta_block(ramp_topology,ramp_shape,meta_shape)
```
where the possible return values of ```meta_block_info``` are:
    * ```None``` if no such block can be found
    * ```(meta_block, meta_block_shape, meta_block_origin)``` where
        * ```meta_block``` is a list of 3-tuple server ids that exist in RAMP and will be reserved for this meta block
        * ```meta_block_shape``` is a 3-tuple referring to the shape of the meta-block w.r.t. RAMP dimensions
        * ```meta_block_origin``` is a 3-tuple identifying an effective top-left corner of this meta-block (though 'top-left' not exactly accurate since RAMP follows PAC-man rules).
* If ```meta_block_info is not None``` then call for an allocation attempt to take place
```python
allocated = GreedyBlockAllocator.allocate(ramp_topology,graph,sequence,splits,meta_block_info,parents,op_server_info)
```
where the possible return values of ```allocated``` are:
    * ```None``` if the allocation was not possible (i.e. resources for some op could not be found during allocation)
    * ```(ramp_topology, op_server_info)``` where both of these values are as described earlier, but updated w.r.t. the resources allocated
    
* If allocation was successful, then update the relevant parameters;
```python
ramp_topology, op_server_info = allocated
```

## Pseudo-code:

```python
# required inputs (handled outside of allocation logic)
job_graph = get_graph_from_wherever()
mp_split_ids, mp_splits = get_splits_from_wherever(job_graph)
meta_block_shape = get_meta_block_shape_from_agent(job_graph)
ramp_topology, ramp_shape = get_ramp_details_from_wherever()

# get the allocation 'preamble' (job info used for the heuristic allocator)
sequence, splits, op_server_info, parents, children = GreedyBlockAllocator.get_allocation_preamble(job_graph,mp_split_ids,mp_splits)

# get a meta-block of a particular shape which the heuristic allocator will try to pack the job fully into
meta_block_info = GreedyBlockAllocator.find_meta_block(ramp_topology,ramp_shape,meta_shape)

# if a meta-block was successfully found...
if meta_block_info:
    # try to allocate the job
    allocated = GreedyBlockAllocator.allocate(ramp_topology,graph,sequence,splits,meta_block_info,parents,op_server_info)
    # if the allocation was successful...
    if allocated:
        # update the topology and op-server info for use in the next job allocation
        ramp_topology, op_server_info = allocated
```


# Notes on meta-block allocation: allocating blocks of resources that all ops for a single job will be packed into)

* The meta block is allocated w.r.t. a specific shape that is given by some agent (or whatever). This shape is found in the RAMP topology when servers can be found that have no ops currently using them (i.e. different jobs cannot share servers, but ops within the same job can).
* PAC-man type rules apply to the search process, where if a shape search exceedes some boundary it will continue on the opposite end (i.e. the loop that iterates over RAMP dimensions is done using modulo so that dimensions are never exceeded).

# General questions

* [George/Alessandro] does it only make sense to put children on same servers as a parent if N(parent sub-ops) == N(children sub-ops)?
    * If more children, then you are effectively undermining the partitioning attempt (e.g. 4 sub-ops put in 2 servers)
    * If less children, is this a collective?
        * I think no becuase if e.g. 4 servers where children are only on 2 of them, than there is an uneven message passing.

In [29]:
%load_ext autoreload

from collections import deque
from copy import deepcopy

def dummy_ramp(shape):
    '''
    Generates a dummy ramp 'network' which is actually
    just a dictionary. This is fine though, since the
    topology of ramp is not really interacted with in 
    any case.
    '''
    c = shape[0]
    r = shape[1]
    s = shape[2]
    ramp = {}
    for i in range(c):
        for j in range(r):
            for k in range(s):
                ramp[(i,j,k)] = {'mem':1,'ops':[]}

    return ramp

class GreedyBlockAllocator:
    '''
    NOTE: These are the following simplifications to make the heuristic doable:
            - blocks are only constructed in a 'connected' way (i.e. if you need 
            1 server per rack for 2 racks and each has to be on a different
            communication group, the groups will be checked as 1-2, 2-3 etc not
            1-6, 2-5 and so on. This is to avoid exhaustive searches.
            - everything is handled in multiples of 2 (i.e. it is assumed that
            ops are partitioned to a multiple of 2 number of sub ops etc). This 
            keeps things simple w.r.t. symmetries since an odd number of servers
            can only be placed on RAMP in a more constrained way if the collective
            symmetry rules are to be followed, so this keeps things simple.
    '''
    
    def __init__(self):
        pass

    @staticmethod
    def _get_factor_pairs(n):
        '''
        This function returns a list of tuples specifying
        all integer factor-pairs for a given integer, n.

        This is used for finding symmetric server-blocks 
        to be allocated for collective communication, where
        before a block can be found, the possible 'shapes' 
        of block (given a number of servers required) need to be known.
        '''
        pairs = []

        for i in range(1,n+1):

            if n % i == 0:
                pairs.append((int(n/i),i))

        return pairs



    @staticmethod
    def _check_block(ramp,block,op_size,mode):
        '''
        Iterate through each server in a block.

        Return True if each server has enough resource
        to support the requirement, False otherwise
        '''
        if block == []:
            return False
        for server in block:
            if mode == 'sub':
                if ramp[server]['mem'] < op_size:
                    return False
            if mode == 'meta':
                if ramp[server]['ops'] != []:
                    return False
        return True
    
    @staticmethod
    def _get_meta_block(C,R,S,ramp_shape,origin=(0,0,0)):
        '''
        Given an origin (i.e. a starting server in RAMP), 
        returns a set of servers that are part of a 'shape'
        that is centred at that origin. 

        NOTE: A simplification here is that in the case of
        one server-per-rack, servers of the same id are checked.
        If this is not done, then a fully exhaustive search has to
        be implemented which is infeasible (i.e. for group of racks
        with one server each, every possible combination of servers
        across racks has to be checked...).
        '''
        block = []
        i,j,k = origin

        for c in range(C):
            for r in range(R):
                for s in range(S):
                    block.append(((i+c)%ramp_shape[0],(j+r)%ramp_shape[1],(k+s)%ramp_shape[2]))

        return block
    
    @staticmethod
    def _ff_meta_block(block_shapes,ramp_shape,ramp,mode,op_size=None,meta_block_origin=(0,0,0)):
        '''
        For each block shape, create the block.

        When a block has been created, check it using
        check_block to see if it's OK for allocation
        w.r.t. server-resources. If OK, return the block.
        Otherwise, return None.

        The seach will start at the point 'meta_block_origin' which
        should be the upper left hand corner of the block that has
        been allocated to the job. This function will then find a
        sub-block that can be given to a particular (partitioned) 
        op in that job.

        NOTE: currently this code is using (0,0,0) as the upper left
        hand corner of the meta-block. This will have to be an argument
        as different block throughout allocation will have different 
        positions in the RAMP network.

        NOTE: all functions here are only working in multiples of 2.
        this is because accounting for odd server-numbers means extra 
        conditions to be handled when distributing over multiple racks
        because of the RAMP symmetry rules for collectives.
        '''

        orgn_c, orgn_r, orgn_s = meta_block_origin
        for shape in block_shapes:
            #get the acceptable search ranges given how big the meta-block is
            #all shapes will already be maximum the size of the meta-block, so this doesn't have to be checked
            I = ramp_shape[0]-shape[0]+1
            J = ramp_shape[1]-shape[1]+1
            K = ramp_shape[2]-shape[2]+1
            if I <= 0 or J <= 0 or K <= 0:
                continue
            else:
                #get the size of the shape in each RAMP dimension
                C,R,S = shape
                for i in range(ramp_shape[0]):
                    # j = i
                    for j in range(ramp_shape[1]):
                        for k in range(ramp_shape[2]):
                            block = GreedyBlockAllocator._get_meta_block(C,R,S,ramp_shape,origin=((orgn_c+i),(orgn_r+j),(orgn_s+k)))

                            if GreedyBlockAllocator._check_block(ramp,block,op_size,mode):
                                if mode == 'sub':
                                    return block
                                if mode == 'meta':
                                    return (block, shape, (orgn_c+i,orgn_r+j,orgn_s+k))

        return None
    
    @staticmethod
    def _ff_block(block_shapes,meta_shape,ramp_shape,ramp,mode,op_size=None,meta_block_origin=(0,0,0)):
        '''
        For each block shape, create the block.

        When a block has been created, check it using
        check_block to see if it's OK for allocation
        w.r.t. server-resources. If OK, return the block.
        Otherwise, return None.

        The seach will start at the point 'meta_block_origin' which
        should be the upper left hand corner of the block that has
        been allocated to the job. This function will then find a
        sub-block that can be given to a particular (partitioned) 
        op in that job.

        NOTE: currently this code is using (0,0,0) as the upper left
        hand corner of the meta-block. This will have to be an argument
        as different block throughout allocation will have different 
        positions in the RAMP network.

        NOTE: all functions here are only working in multiples of 2.
        this is because accounting for odd server-numbers means extra 
        conditions to be handled when distributing over multiple racks
        because of the RAMP symmetry rules for collectives.
        '''
        # print(f'find_sub_block: {ramp.keys()}')
        orgn_c, orgn_r, orgn_s = meta_block_origin
        for shape in block_shapes:
            #get the acceptable search ranges given how big the meta-block is
            #all shapes will already be maximum the size of the meta-block, so this doesn't have to be checked
            I = (meta_shape[0]-shape[0])+1
            J = (meta_shape[1]-shape[1])+1
            K = (meta_shape[2]-shape[2])+1
            if I <= 0 or J <= 0 or K <= 0:
                continue
            else:
                #get the size of the shape in each RAMP dimension
                C,R,S = shape

                for i in range(I):
                    # j = i
                    for j in range(J):
                        for k in range(K):
                            #get a block of shape (C,R,S) at origin (i,j,k)
                            block = GreedyBlockAllocator._get_block(C,R,S,ramp_shape,origin=(orgn_c+i,orgn_r+j,orgn_s+k))
                            if GreedyBlockAllocator._check_block(ramp,block,op_size,mode):
                                if mode == 'sub':
                                    return block
                                if mode == 'meta':
                                    return (block, shape, (orgn_c+i,orgn_r+j,orgn_s+k))

        return None
    
    @staticmethod
    def _get_block(C,R,S,ramp_shape,origin=(0,0,0)):
        '''
        Given an origin (i.e. a starting server in RAMP), 
        returns a set of servers that are part of a 'shape'
        that is centred at that origin. 

        NOTE: A simplification here is that in the case of
        one server-per-rack, servers of the same id are checked.
        If this is not done, then a fully exhaustive search has to
        be implemented which is infeasible (i.e. for group of racks
        with one server each, every possible combination of servers
        across racks has to be checked...).
        '''
        block = []
        i,j,k = origin

        if S == -1:
            for n in range(C):
                block.append(((i+n)%(ramp_shape[0]+1),(j+n)%(ramp_shape[1]+1),k))
        else:
            for c in range(C):
                for r in range(R):
                    for s in range(S):
                        block.append(((i+c)%(ramp_shape[0]),(j+r)%(ramp_shape[1]),(k+s)%(ramp_shape[2])))
        return block
    
    @staticmethod
    def find_meta_block(ramp_topology,ramp_shape,meta_block_shape):
        '''
        Take an input of a RAMP topology and a required number of servers and 
        return a block of servers that is symmetric w.r.t. collective and where
        every server is currently empty.
        
        Returned values should be a set of server IDs, the shape and the 'origin'
        server in that set (i.e. the effective top left hand corner).
        
        meta_block_info = (meta_block, meta_block_shape, meta_block_origin) or None if empty.
        '''
        meta_block_info = GreedyBlockAllocator._ff_meta_block([meta_block_shape],ramp_shape,ramp_topology,'meta')
        # print(f'find_meta_block: {ramp_topology.keys()}')
        return meta_block_info 
    
    @staticmethod
    def find_sub_block(ramp_topology,ramp_shape,meta_block_shape,meta_block_origin,num_servers,op_size):
        
        pairs = GreedyBlockAllocator._get_factor_pairs(num_servers)
        block_shapes = GreedyBlockAllocator._get_block_shapes(pairs,meta_block_shape)
        #if no possible shapes try rack and CG distributed
        block_shapes += [(num_servers,num_servers,-1),(num_servers,1,1)]
        # print(f'find_sub_block: {ramp_topology.keys()}')
        block = GreedyBlockAllocator._ff_block(block_shapes,meta_block_shape,ramp_shape,ramp_topology,'sub',op_size=op_size,meta_block_origin=meta_block_origin)
        return block
    
    @staticmethod
    def _get_parents_and_children(original_graph):
        
        parents = {}
        children = {}
        tmp_graph = original_graph.copy()
        
        for node in original_graph.nodes():
                in_nodes = [in_edge[0] for in_edge in original_graph.in_edges(node)]
                parents[node] = in_nodes
                
                out_nodes = [out_edge[1] for out_edge in original_graph.out_edges(node)]
                children[node] = out_nodes
                
        return parents, children
    
    @staticmethod
    def _topo_sort(parents,children):
                
        sequence = []
        
        queue = deque()
        
        for node in parents.keys():
            if parents[node] == []:
                queue.append(node)
                sequence.append(node)
                
        while len(queue) > 0:
            
            node = queue.popleft()
            for child in children[node]:
                parents[child].remove(node)
                if parents[child] == []:
                    queue.append(child)
                    sequence.append(child)
                    
        return sequence
        
    @staticmethod
    def _regular_collective_placement(ramp,ramp_shape,job_graph,op,split,meta_block_info,op_server_info):
        '''
        This function allocates a split op to a set of servers in a meta-block nd returns a dictionary 
        of which sub-ops are allocated to which servers, and another dictionary indicating across which
        servers are each op distributed (this does not refer to specific sub-ops and is used so that 
        the parent-checking allocation method can be implemented. Sub-ops are allocated one op per
        server.
        
        Args:
            ramp (dict): RAMP 'topology' as a dict like {(a,b,c):attributes}
            job_graph (nx.DiGraph): un-partitioned and non-mirrored computational graph of a job
            op (str): name of an op (e.g. '11')
            split (int): how many times this op should be split
            meta_block_info (tuple): return value of GreedyBlockAllocator.find_meta_block
            op_server_info (dict): which ops are allocated across which servers (e.g. {'11':[(0,0,1),(0,0,2)]})
                                    
        '''
        # print(f'_regular_collective_placement: {ramp.keys()}')
        num_nodes = len(job_graph.nodes())
        meta_block,meta_block_shape,meta_block_origin = meta_block_info

        num_servers = split #NOTE: ops_per_server should never be more than splits. Needs to be ensured somewhere. EDIT: since putting multiple sub-ops on the same server is trivial (should then just partition it by a smaller amount) we will keep 1 op-per-server for now.
        if num_servers > len(meta_block): #if there are fewer servers in the meta-block than asked for, no allocation
            return None

        op_size = job_graph.nodes[op]['activation']/split
        meta_block = {server:ramp[server] for server in meta_block}

        block = GreedyBlockAllocator.find_sub_block(ramp,ramp_shape,meta_block_shape,meta_block_origin,num_servers,op_size)

        if not block: #if no block can be found (memory errors) then return None
            return None
        for j in range(len(block)):
            ramp[block[j]]['mem'] -= op_size
            if split > 1:
                ramp[block[j]]['ops'].append(op+chr(97+j))
                ramp[block[j]]['ops'].append(str(int(op)+num_nodes)+chr(97+j))
            else:
                ramp[block[j]]['ops'].append(op)
                ramp[block[j]]['ops'].append(str(int(op)+num_nodes))
            op_server_info[op].append(block[j])

        #if allocation was feasible, return the updated (i.e. with server-memory reduced) RAMP topology for further allocations
        return ramp, op_server_info
    
    @staticmethod
    def _parent_collective_placement(ramp,job_graph,op,split,meta_block_info,parents,op_server_info):
        '''
        Similar to _regular_collective_placement except it checks if the given op's
        parents are allocated already somewhere. It then checks if it can allocate
        all the sub-ops evenly across the set of servers associated with (one of) the
        parent(s). If there are more child sub-ops than servers used by the parent, then 
        they are packed in evenly across the servers. 
        
        Args:
            ramp (dict): RAMP 'topology' as a dict like {(a,b,c):attributes}
            job_graph (nx.DiGraph): un-partitioned and non-mirrored computational graph of a job
            op (str): name of an op (e.g. '11')
            split (int): how many times this op should be split
            parents (dict): information relating ops in job_graph to their parents (i.e. {'14':['12','13']} if 12 and 13 are parents of 14).
            op_server_info (dict): which ops are allocated across which servers (e.g. {'11':[(0,0,1),(0,0,2)]})
        '''
        # #can't split an un-split node over it's split parents
        # if split == 1:
        #     return None
        
        op_requirement = job_graph.nodes[op]['activation']
        num_nodes = len(job_graph.nodes())
        
        #get sets of servers corresponding to each parent
        parents_servers = []
        for parent in parents[op]:
            if set(op_server_info[parent]).issubset(meta_block_info[0]):
                parents_servers.append(op_server_info[parent])

        #for each set of servers
        for servers in parents_servers:
            if split < len(servers):
                continue
            else:
                #check if ops can fit evenly across the servers
                available_resource = sum([ramp[server]['mem'] for server in servers])
                if available_resource >= op_requirement:
                    i = 0
                    while i < split:
                        for server in servers:
                            ramp[server]['mem'] -= op_requirement/split
                            # if split > 1:
                            ramp[server]['ops'].append(op+chr(97+i))
                            ramp[server]['ops'].append(str(int(op)+num_nodes)+chr(97+i))
                            op_server_info[op].append(server)
                            i += 1
                    return ramp, op_server_info

        
        return None
    
    @staticmethod
    def allocate(ramp,ramp_shape,job_graph,sequence,splits,meta_block_info,parents,op_server_info):
        
        # op_server_info = {op:[] for op in job_graph.nodes()}
        # print(f'allocate: {ramp.keys()}')
        for i in range(len(sequence)):
            op = sequence[i]
            split = splits[i]
            
            #try allocating on same servers as parents
            alloc = GreedyBlockAllocator._parent_collective_placement(ramp,job_graph,op,split,meta_block_info,parents,op_server_info)
            
            #if this doesn't work, try allocating somewhere else
            if not alloc:
                # print('no parents - regular allocation')
                alloc = GreedyBlockAllocator._regular_collective_placement(ramp,ramp_shape,job_graph,op,split,meta_block_info,op_server_info)
                
            #if that didn't work either, return None (this means the allocation has failed)
            if not alloc:
                return alloc
            
            #if either of the allocation attempts worked, update ramp and op_server_info and go onto the next op
            else:
                ramp, op_server_info = alloc[0], alloc[1]
                
        return ramp, op_server_info
    
    @staticmethod
    def get_allocation_preamble(job_graph,mp_split_ids,mp_splits):
        
        parents, children = GreedyBlockAllocator._get_parents_and_children(job_graph)
        sequence = GreedyBlockAllocator._topo_sort(deepcopy(parents),deepcopy(children))
        op_server_info = {op:[] for op in job_graph.nodes()}
        splits = []
        for s in sequence:
            if int(s) in mp_split_ids:
                idx = mp_split_ids.index(int(s))
                splits.append(mp_splits[idx])
            else:
                splits.append(1)
        return sequence, splits, op_server_info, parents, children
    
    @staticmethod
    def _get_block_shapes(pairs,ramp_shape):
        '''
        Given a set of factor-pairs (corresponding to a 
        particular number of servers that have to be 
        allocated into a block) and the size of the full 
        ramp meta-block that is to have the job packed
        into it, return the set of acceptable 'shapes'
        of block that can fit within the size of this 
        meta-block.
        '''
        blocks = []
        # print(ramp_shape)
        for pair in pairs:
            if pair[0] > ramp_shape[0] or pair[0] > ramp_shape[1] or pair[1] > ramp_shape[2]:
                continue
            else:
                # blocks.append((pair[0],pair[0],pair[1]))
                blocks.append((pair[0],1,pair[1]))
                blocks.append((pair[0],pair[1],1))
        
        return blocks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Partition: Partitioning DAGs from pipedream

## General description

* The ```Partition``` class allows for graphs to be read from pipedream files and re-drawn as partitioned (model and/or data) graphs
* Arguments (__init__):
    * ```graph``` (nx.DiGraph): Represents a DAG. These must be from pipedream file and read specifically using the Partition.pipedream_graph static-method. 
    * ```mp_split_ids``` (list[int]): A list of integers corresponding to the IDs of ops in the graph (which are also integer numbered) where the included IDs refer to the ops in the DAG that are going to be (model) partitioned. 
    * ```mp_splits``` (list[int]): A list of integers corresponding to the number of times an op will be split, where the indices of these values correspond to the indices in ```mp_split_ids```
    * dp_splits (int, default=0): The number of times the model should be copied (i.e. for data-partitioning). This should not be used for now. 
* High level implementation details (excluding data-partitioning for now):
    1. A mirror version of the graph is created and attached to the end of the original graph. 
        * This is so that the backward pass is explicitly represented in the graph when followed 'down' from source to sink via the dependencies.
    2. Each node that is to be partitioned is split. 
        * The naming convention followed is that node IDs are strings of integers and post-partitioned sub-ops are appended with letters to distinguish them from each other.
            * e.g. node '1' split 2 times will become {'1a','1b'}
        * This process is applied simultaneously to both forward and backward pass 'versions' of each op. 
            * For the backward pass 'versions' of the partitioned op, an additional step is that an all to all connectivity structure is established between them. This is because on a backward pass, all sub-ops must coordinate their gradients etc with each other so collective communication is required, which in this case also requires all-to-all.
    
* Additional notes:
    * Initialising a ```Partition``` object will return a single new graph which has the above process applied to it. 
    * A ```Partition``` object also has a ```self.original_graph``` attribute, which is just the graph as returned from ```Partition.pipedream_graph``` (i.e. the un-partitioned, un-mirrored version).  
    
## Pseudo-code:

```python
# load the networkx representation of the pipedream graph
job_graph = Partition.pipedream_graph(PATH_TO_PIPEDREAM_FILE)

# op '1' will be split into 2 sub-ops
mp_split_ids, mp_splits = [1],[2]

# get the partitioned graph
partition_job_graph = Partition(job_graph,[6],[2])
```

In [30]:
%load_ext autoreload

from collections import deque
from copy import deepcopy
import networkx as nx

class Partition:
    
    def __init__(self,graph,mp_split_ids,mp_splits,dp_splits=0):
        '''
        high level implementation explanation:
            1. take the forward only graph and create a forward + backward version of it
            2. data partition the model dp_splits times
                a. all output nodes are currently connected to each other (this needs to be reconsidered)
                b. 'data' edge attribute is set to the size of 'activation' on the src node
            3. model partition the nodes in mp_split_ids (list) to subnodes equal to same-index values in mp_splits (list)
                a. for each sub-node for node mp_split_ids[i], the incoming 'data' attribute on the incoming edge is set to 'data'/mp_splits[i]. This is done since when ops are split, the amount of information going into and out of each op (into the next op) is split.
        '''
        
        self.mp_split_ids = mp_split_ids
        self.mp_splits = mp_splits
        self.dp_splits = dp_splits
        self.original_graph = graph
        backward = self.mirror_graph(graph)
        self.graph = self.combine_graphs(graph,backward)
        
        self.data_split_node()
        self.model_split_node()
        
    def mirror_graph(self,graph):

        forward_graph = graph.copy()
        n = len(forward_graph.nodes())
        nodes = []

        for i in range(1,len(forward_graph.nodes())+1):

            nodes.append((str(2*n-(i-1)),forward_graph.nodes[str(i)]))

        edges = list(forward_graph.edges())

        for i in range(len(edges)):

            edges[i] = (str(2*n-(int(edges[i][1])-1)),str(2*n-(int(edges[i][0])-1)))

        backward_graph = nx.DiGraph()
        backward_graph.add_nodes_from(nodes)
        backward_graph.add_edges_from(edges)

        return backward_graph

    def combine_graphs(self,forward,backward):

        forward_nodes = list(forward.nodes())
        backward_nodes = list(backward.nodes())

        for i in list(forward.nodes()):
            forward.nodes[str(i)]['compute'] = forward.nodes[str(i)]['forward']
            del forward.nodes[str(i)]['forward']
            del forward.nodes[str(i)]['backward']

        for i in list(backward.nodes()):
            backward.nodes[str(i)]['compute'] = backward.nodes[str(i)]['backward']
            del backward.nodes[str(i)]['forward']
            del backward.nodes[str(i)]['backward']

        join_0, join_1 = max([int(nd) for nd in forward_nodes]),min([int(nd) for nd in backward_nodes])
        joined = nx.union(forward,backward)

        joined.add_edge(str(join_0),str(join_1))

        for edge in joined.edges():
            edge = tuple(edge)
            joined.edges[edge[0],edge[1]]['communication'] = joined.nodes[edge[0]]['activation']

        return joined
    
    def data_split_node(self):

        '''goal: 
        - create n_splits copies of a given graph with no repeated node ids
        - do this before node splits, so that the same set of nodes splits can be applied to all of these graphs at once
        '''

        og_nodes = [int(node) for node in list(self.graph.nodes())]
        og_edges = [(int(edge[0]),int(edge[1])) for edge in self.graph.edges()]

        self.highest_og_node = max(og_nodes)

        all_highest_nodes = []

        #currently assuming that data splitting doesn't require model size to change in any way
        new_features = [self.graph.nodes[str(node)] for node in og_nodes]

        new_graph = nx.DiGraph()

        for i in range(self.dp_splits+1):
            id_shift = i*self.highest_og_node
            new_nodes = [(str(og_nodes[j]+id_shift),new_features[j]) for j in range(len(og_nodes))]
            new_edges = [(str(edge[0]+id_shift),str(edge[1]+id_shift)) for edge in og_edges]

            all_highest_nodes.append(self.highest_og_node+id_shift)

            new_graph.add_nodes_from(new_nodes)
            new_graph.add_edges_from(new_edges)

        edge_features = {}
        for edge in new_graph.edges():
            edge_features[edge] = {'data':new_graph.nodes[edge[0]]['activation']}
            
        nx.set_edge_attributes(new_graph,edge_features)
        
        self.graph = new_graph

    def model_split_node(self):

        in_edge_features = {}
        out_edge_features = {}
        
        #do for each op that should be split
        for i in range(len(self.mp_split_ids)):

            n_splits = self.mp_splits[i]
            #do across each data-parallel split graph
            for j in range(self.dp_splits+1):
                node_ids = [
                            str(self.mp_split_ids[i]+(j*self.highest_og_node)),
                            str(self.highest_og_node - (self.mp_split_ids[i]-1)+(j*self.highest_og_node))
                           ]
                            
                for k in range(len(node_ids)):
                    
                    node_id = node_ids[k]
                    
                    in_edges = [edge[0] for edge in self.graph.in_edges(node_id)]
                    out_edges = [edge[1] for edge in self.graph.out_edges(node_id)]

                    new_feature = {'activation':self.graph.nodes[node_id]['activation']/n_splits,
                                   'parameter':self.graph.nodes[node_id]['parameter']/n_splits,
                                   'compute':self.graph.nodes[node_id]['compute']/n_splits,
                                   'type':self.graph.nodes[node_id]['type']
                                  }

                    nodes = [('{}{}'.format(node_id,chr(97+k)),new_feature) for k in range(n_splits)]

                    new_edges = []
                    for node in nodes:
                        new_edges += [(edge,node[0]) for edge in in_edges]
                        new_edges += [(node[0],edge) for edge in out_edges]

                        #account for edge features for incoming edges to split nodes
                        for edge in in_edges:
                            in_edge_features[(edge,node[0])] = {'data':self.graph.nodes[edge]['activation']/len(nodes)}
                            
                    #extra step if on the back-prop for collective between all sub-ops to sync weights
                    if k == 1:
                        for l in range(len(nodes)):
                            for m in range(len(nodes)):
                                if l == m:
                                    continue
                                new_edges += [(nodes[l][0],nodes[m][0])]

                    self.graph.remove_node(node_id)
                    self.graph.add_nodes_from(nodes)
                    self.graph.add_edges_from(new_edges)
                
        nx.set_edge_attributes(self.graph,in_edge_features)
      
    @staticmethod
    def pipedream_graph(file_path:str):

        graph = nx.DiGraph()

        f = list(open(file_path,'r'))

        nodes = []
        edges = []

        for line in f:

            line = line.split(' -- ')
            for idx, el in enumerate(line):
                line[idx] = el.split('\t')[-1]

            #if this line represents a node
            if len(line) > 2:

                node_features = {}

                #get node id
                node_id = str(int(line[0][4:]))

                #get op id
                op_id = line[1].split('(')[0]

                node_features['type'] = op_id

                #get op details
                op_details = str(line[1].split(op_id)[1][1:-1]).split(', ')


                #get compute time and memory details
                comp_and_memory = line[2].split(', ')
                comp_memory_feats = ['forward','backward','activation','parameter']
                for i in range(len(comp_memory_feats)):
                    node_features[comp_memory_feats[i]] = float(comp_and_memory[i].split('=')[1].replace('\n',''))

                nodes.append((node_id,node_features))
            else:

                src = int(line[0][4:])
                dst = int(line[1][4:])

                edges.append((str(src),str(dst))) #assume only 1 data channel for now

        #get initial graph
        graph.add_nodes_from(nodes)
        graph.add_edges_from(edges)

        return graph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# required inputs (handled outside of allocation logic)
job_graph = Partition.pipedream_graph('/scratch/datasets/ddls/jobs/pipedream_graphs/image_classification/profiles/alexnet/graph.txt') #REPLACE_THIS_WITH_A_REAL_PATH_TO_A_PIPEDREAM_FILE)

# Pipedream ops have large numbers. For debugging etc it's easier to have unit sized resources so this is forced here. Not necessary in real implementation.
for node in job_graph.nodes():
    job_graph.nodes[node]['activation'] = 2.0

# specify which ops are going to be split and how many times
mp_split_ids, mp_splits = [1,2],[2,4]
partition_job_graph = Partition(job_graph,[6],[2])

# specify size of the meta-block to be used for this job.
meta_shape = (2,2,2)

# specify the RAMP topology (this is using a simple dict-based method, can be implemented in some other way in theory). 
ramp_shape = (4,2,4)
ramp_topology = dummy_ramp(ramp_shape)
# print(f'script: {ramp_topology.keys()}')
# get the allocation 'preamble' (job info used for the heuristic allocator)
sequence, splits, op_server_info, parents, children = GreedyBlockAllocator.get_allocation_preamble(job_graph,mp_split_ids,mp_splits)

# Only use the first 2 ops in the graph. This is also for debugging purposes (using very small RAMPs is easy to visualise etc). These two lines would not be used in real thing. 
sequence = sequence[:2]
splits = splits[:2]

# get a meta-block of a particular shape which the heuristic allocator will try to pack the job fully into
meta_block_info = GreedyBlockAllocator.find_meta_block(ramp_topology,ramp_shape,meta_shape)

# if a meta-block was successfully found...
if meta_block_info:
    print('meta block found')
    # simple visualisation of the RAMP topology, where servers marked 'x' are those included in the meta-block (i.e. those reserved for this job). 
    image = ''
    for c in range(ramp_shape[0]):
        for r in range(ramp_shape[1]):
            for s in range(ramp_shape[2]):
                if (c,r,s) in meta_block_info[0]:
                    image += '[x]'
                else:
                    image += '[ ]'
            image += '  '
        image += '\n'
    print(image)

    # try to allocate the job
    allocated = GreedyBlockAllocator.allocate(ramp_topology,ramp_shape,job_graph,sequence,splits,meta_block_info,parents,op_server_info)
    # if the allocation was successful...
    if allocated:
        # print out which servers were allocated to which ops
        print(f'allocation found:')
        for k,v in allocated[1].items():
            if v:
                print(f'op:{k} - servers:{v}')
        # update the topology and op-server info for use in the next job allocation
        ramp_topology, op_server_info = allocated

meta block found
[x][x][ ][ ]  [x][x][ ][ ]  
[x][x][ ][ ]  [x][x][ ][ ]  
[ ][ ][ ][ ]  [ ][ ][ ][ ]  
[ ][ ][ ][ ]  [ ][ ][ ][ ]  

allocation found:
op:1 - servers:[(0, 0, 0), (1, 0, 0)]
op:2 - servers:[(0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1)]
